# Практика
Используемые библиотеки

In [3]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [4]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)
df['Норматив'].unique()

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


array(['в пределах 6-9', 'не более 2-3', 'не более 30', 'в пределах 7-10',
       'не более 1,5', 'не более 3,3', 'не более 45', 'не более 3,5',
       'не более 350', 'не более 500', 'не более 0,3', 'не более 1000',
       'не более 0,5'], dtype=object)

In [5]:
# prepare values to normal value
df.loc[df['Результат анализа'] == 'б/цвета', 'Результат анализа'] = 0
df.loc[df['Норматив'] == 'не более 2-3', 'Норматив'] = 'не более 3'
df['Результат анализа'] = df['Результат анализа'].astype(float)
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.400,в пределах 6-9
1,Запах,баллы,1.000,не более 3
2,Цветность,градусы,0.000,не более 30
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24.000,не более 45
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200.000,не более 350
9,Сульфаты (SO4),мг/дм3,189.500,не более 500


In [6]:
# Ваше решение
import numpy as np
# replace ',' with '.' to apply pattern
df['Норматив'] = df['Норматив'].str.replace(',', '.')
# add min and max value for normativ
df[['min', 'max']] = df.Норматив.str.findall('(\d*\.?\d+)').apply(lambda x: x if len(x) == 2 else [None, x[0]]).values.tolist()
df[['min', 'max']] = df[['min', 'max']].astype(float)

In [7]:
def get_is_normal(min_, max_, val):
    return min_ <= val <= max_ if not pd.isna(min_) else val <= max_

df['is_normal'] = df[['min', 'max', 'Результат анализа']].apply(lambda x: get_is_normal(*x), axis=1)
df = df.set_index('Показатель')
df

,Единица измерений,Результат анализа,Норматив,min,max,is_normal
Показатель,,,,,,
pH,единицы pH,8.400,в пределах 6-9,6.0,9.0,True
Запах,баллы,1.000,не более 3,NaN,3.0,True
Цветность,градусы,0.000,не более 30,NaN,30.0,True
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7.0,10.0,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,NaN,1.5,True
Нитриты (по NO2),мг/дм3,0.017,не более 3.3,NaN,3.3,True
Нитраты (по NO3),мг/дм3,24.000,не более 45,NaN,45.0,True
Фосфаты (P),мг/дм3,0.360,не более 3.5,NaN,3.5,True
Хлориды (Cl),мг/дм3,200.000,не более 350,NaN,350.0,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [8]:
import random
APPLE = 0
ORANGE = 1
basket = [APPLE] * 4 + [ORANGE] * 5
random.shuffle(basket)
print(basket)

res = []
count = 1000000
res = [ORANGE for _ in range(count) if set(random.sample(basket, 3)) == {ORANGE}]
len(res) / count

[1, 1, 1, 0, 1, 0, 1, 0, 0]


0.118585

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [22]:
STANDARD = 0
NON_STANDARD = 1
basket = [STANDARD] * 7 + [NON_STANDARD] * 3
random.shuffle(basket)
print(basket)
count = 100000

# проверим, что первая деталь - нестандартная, а вторая - стандартная
result = [True for _ in range(count) if random.sample(basket, 2) == [NON_STANDARD, STANDARD]]

len(result) / count

[0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


0.23316